In [1]:
import pandas as pd

df_clients = pd.read_csv('./content/caso3/clients.csv')
df_depots = pd.read_csv('./content/caso3/depots.csv')
df_vehicles = pd.read_csv('./content/caso3/vehicles.csv')

In [2]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [3]:
precio_km = 3032.1

# Leer el CSV
df = pd.read_csv('distances_all.csv')

# Inicializar ambos diccionarios
distance = {}
cost     = {}

# Recorrer una única vez (usando zip para eficiencia)
for ori, dst, d in zip(df['origen'], df['destino'], df['distancia_km']):
    key = (ori, dst)
    distance[key] = d
    cost[key]     = d * precio_km

# Verificar
print(distance[('CD1','CD2')], cost[('CD1','CD2')])

32.99 100028.979


In [4]:
depot_capacity = {}
for depot, d in zip(df_depots['DepotID'], df_depots['Capacity']):
    depot_capacity[f'CD{depot}'] = d


demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c


vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c


vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

In [5]:
# Etiquetas
depot_ids    = [str(d) for d in df_depots['DepotID']]
depot_labels = [f"CD{d}" for d in depot_ids]
client_ids   = [str(c) for c in df_clients['ClientID']]
client_labels= [f"C{c}"  for c in client_ids]
vehicle_list = df_vehicles['VehicleName'].tolist()

In [6]:
unassigned = set(client_labels)
cientesdeposito = {d: [] for d in depot_labels}

# radios a probar:
radii = sorted({
    dist
    for (d, c), dist in distance.items()
    if d in depot_labels and c in client_labels
})

for R in radii:

    for d in depot_labels:
        totaldemanda = sum(demand[c] for c in cientesdeposito[d])
        available_capacity = depot_capacity[d] - totaldemanda

        if available_capacity <= 0:
            continue

        for c in list(unassigned):
            if distance[(d, c)] <= R and demand[c] <= available_capacity:
                cientesdeposito[d].append(c)
                unassigned.remove(c)
                available_capacity -= demand[c]
        if not unassigned:
            break

    if not unassigned:
        break  # si todos los clientes están asignados, salir del ciclo de los radios

# 4) Resultado
for d, clist in cientesdeposito.items():
    print(f"{d} atiende a {len(clist)} clientes: {clist}")


CD1 atiende a 0 clientes: []
CD2 atiende a 0 clientes: []
CD3 atiende a 5 clientes: ['C31', 'C77', 'C5', 'C6', 'C2']
CD4 atiende a 12 clientes: ['C7', 'C67', 'C74', 'C58', 'C66', 'C69', 'C4', 'C47', 'C20', 'C63', 'C41', 'C82']
CD5 atiende a 14 clientes: ['C48', 'C16', 'C45', 'C18', 'C3', 'C51', 'C28', 'C17', 'C60', 'C46', 'C76', 'C65', 'C19', 'C42']
CD6 atiende a 13 clientes: ['C55', 'C30', 'C85', 'C26', 'C32', 'C57', 'C11', 'C1', 'C13', 'C75', 'C64', 'C87', 'C33']
CD7 atiende a 17 clientes: ['C34', 'C39', 'C15', 'C59', 'C56', 'C23', 'C24', 'C84', 'C35', 'C43', 'C73', 'C89', 'C10', 'C90', 'C14', 'C12', 'C72']
CD8 atiende a 3 clientes: ['C44', 'C62', 'C78']
CD9 atiende a 5 clientes: ['C9', 'C49', 'C61', 'C29', 'C21']
CD10 atiende a 6 clientes: ['C53', 'C25', 'C83', 'C52', 'C80', 'C86']
CD11 atiende a 5 clientes: ['C38', 'C50', 'C40', 'C71', 'C81']
CD12 atiende a 10 clientes: ['C79', 'C22', 'C68', 'C70', 'C37', 'C54', 'C27', 'C88', 'C36', 'C8']


In [7]:
from statistics import mean
import folium
# ————— 1. Mapear etiquetas a coordenadas —————
depot_coords = { f"CD{i+1}": coord_depots[i]
                 for i in range(len(coord_depots)) }
client_coords = { f"C{i+1}": coord_clients[i]
                  for i in range(len(coord_clients)) }

# ————— 2. Centro del mapa —————
all_lons = [lon for lon, lat in coord_depots] + [lon for lon, lat in coord_clients]
all_lats = [lat for lon, lat in coord_depots] + [lat for lon, lat in coord_clients]
map_center = ( mean(all_lats), mean(all_lons) )

# ————— 3. Colores para cada depósito —————
palette = [
    'blue','green','purple','orange','cadetblue','darkred',
    'darkgreen','darkpurple','gray','lightblue','pink','black'
]
colors = {
    depot: palette[i % len(palette)]
    for i, depot in enumerate(depot_coords)
}

# ————— 4. Crear mapa Folium —————
m = folium.Map(location=map_center, zoom_start=12)

# 4.2. Añadir clientes coloreados según su depósito asignado
for depot, client_list in cientesdeposito.items():
    col = colors[depot]
    for client in client_list:
        lon, lat = client_coords[client]
        folium.CircleMarker(
            location=[lat, lon],
            radius=4,
            color=col,
            fill=True,
            fill_color=col,
            fill_opacity=0.7,
            popup=f"{client}, dem={demand[client]}"
        ).add_to(m)

# 4.1. Añadir depósitos con estrella de color según su asignación
for depot, (lon, lat) in depot_coords.items():
    col = colors[depot]   # color idéntico al de sus clientes
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(icon='star', prefix='fa', color=col),
        popup=f"{depot}, cap={depot_capacity[depot]}"
    ).add_to(m)

m

In [8]:
assign_client = {
    c: d
    for d, clist in cientesdeposito.items()
    for c in clist
}

In [9]:
vehicles_per_depot  = {d: [] for d in depot_labels}
assigned_vehicles   = set()
unassigned_vehicles = set(vehicle_list)

for d in depot_labels:
    clientes = cientesdeposito.get(d, [])
    if not clientes:
        continue

    # demanda cliente<capacidad
    totaldemanda = sum(demand[c] for c in clientes)
    if totaldemanda > depot_capacity.get(d, 0):
        print(f"Depósito {d}: capacidad depósito {depot_capacity[d]} < demanda {totaldemanda}")

    # distancia relativa de ir y volver
    total_dist_req = 2 * sum(distance[(d, c)] for c in clientes)

    cap_acum   = 0.0
    range_acum = 0.0

    # vehiculos libres
    libres = sorted(unassigned_vehicles,
                        key=lambda v: vehicle_capacity[v],
                        reverse=True)

    for v in libres:
        if cap_acum >= totaldemanda and range_acum >= total_dist_req:
            break

        vehicles_per_depot[d].append(v)
        assigned_vehicles.add(v)
        unassigned_vehicles.remove(v)

        cap_acum   += vehicle_capacity[v]
        range_acum += vehicle_range[v]

# 5) Mostrar asignaciones
for d, vs in vehicles_per_depot.items():
    if vs:
        print(f"{d} tiene asignados {len(vs)} vehículos (capacidad= {sum(vehicle_capacity[v] for v in vs)}, " +
              f"rango={sum(vehicle_range[v] for v in vs):.1f} km): {vs}")



CD3 tiene asignados 1 vehículos (capacidad= 158, rango=174.0 km): ['V4']
CD4 tiene asignados 2 vehículos (capacidad= 268, rango=342.0 km): ['V2', 'V1']
CD5 tiene asignados 2 vehículos (capacidad= 251, rango=1048.0 km): ['V7', 'V11']
CD6 tiene asignados 2 vehículos (capacidad= 224, rango=280.0 km): ['V3', 'V6']
CD7 tiene asignados 2 vehículos (capacidad= 208, rango=275.0 km): ['V5', 'V10']
CD8 tiene asignados 1 vehículos (capacidad= 98, rango=716.0 km): ['V12']
CD9 tiene asignados 1 vehículos (capacidad= 96, rango=160.0 km): ['V9']
CD10 tiene asignados 1 vehículos (capacidad= 86, rango=1023.0 km): ['V13']
CD11 tiene asignados 1 vehículos (capacidad= 85, rango=942.0 km): ['V15']
CD12 tiene asignados 2 vehículos (capacidad= 153, rango=1210.0 km): ['V8', 'V14']


In [10]:
from pyomo.environ import (ConcreteModel, Set, Param, Var, Objective, Constraint,
                           Binary, NonNegativeReals, minimize, value)
from pyomo.opt import SolverFactory

model = ConcreteModel()

# 2.1) Conjuntos
model.D = Set(initialize=depot_labels)
model.C = Set(initialize=client_labels)
model.V = Set(initialize=vehicle_list)
model.N = model.D | model.C

# 2.2) Parámetros de costo y distancia (ya def. en tu script)
model.cost     = Param(model.N, model.N, initialize=lambda m,i,j: cost[(i,j)],     within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda m,i,j: distance[(i,j)], within=NonNegativeReals)

# 2.3) Capacidades y rangos
model.depot_capacity   = Param(model.D, initialize=lambda m,d: depot_capacity[d])
model.vehicle_capacity = Param(model.V, initialize=lambda m,v: vehicle_capacity[v])
model.vehicle_range    = Param(model.V, initialize=lambda m,v: vehicle_range[v])
model.demand           = Param(model.C, initialize=lambda m,c: demand[c])

# 2.4) Variables
# y[v,d]=1 si v parte del depósito d
model.y = Var(model.V, model.D, domain=Binary)
# x[v,i,j]=1 si v viaja de i a j
model.x = Var(model.V, model.N, model.N, domain=Binary)
# MTZ
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# 2.5) Objetivo: minimizar costo total
def obj_rule(m):
    return sum(m.cost[i,j] * m.x[v,i,j]
               for v in m.V for i in m.N for j in m.N if i!=j)
model.obj = Objective(rule=obj_rule, sense=minimize)

# ──────────────
# 2.6) Restricciones
# a) cada vehículo va a lo sumo a un depósito
def one_depot_per_vehicle(m, v):
    return sum(m.y[v,d] for d in m.D) <= 1
model.one_depot = Constraint(model.V, rule=one_depot_per_vehicle)

# b) si v está asignado a d, sale exactamente una vez de d; si no, no sale
def start_route(m, v, d):
    return sum(m.x[v,d,j] for j in m.N if j!=d) == m.y[v,d]
model.start_route = Constraint(model.V, model.D, rule=start_route)

# d) conservación de flujo en clientes
def flow_cons(m, v, i):
    if i in m.C:
        return ( sum(m.x[v,j,i] for j in m.N if j!=i)
               == sum(m.x[v,i,j] for j in m.N if j!=i) )
    return Constraint.Skip
model.flow_cons = Constraint(model.V, model.N, rule=flow_cons)

def only_own_clients(m, v, i, j):
    # 1) Depósito → Depósito: no imponemos nada
    if i in m.D and j in m.D:
        return Constraint.Skip

    # 2) Arco que termina en cliente j:
    #    solo si v partió de su depósito asignado
    if j in m.C:
        d_j = assign_client[j]
        return m.x[v, i, j] <= m.y[v, d_j]

    # 3) Arco que sale de cliente i → depósito j:
    if i in m.C and j in m.D:
        d_i = assign_client[i]
        return m.x[v, i, j] <= m.y[v, d_i]

    # 4) Cliente → Cliente: MTZ + flujo se encargan, aquí skip
    return Constraint.Skip

model.only_own = Constraint(model.V, model.N, model.N, rule=only_own_clients)


# f) cada cliente lo visita exactamente un vehículo
def each_client_once(m, c):
    return sum(m.x[v,j,c] for v in m.V for j in m.N if j!=c) == 1
model.visit_once = Constraint(model.C, rule=each_client_once)

# g) capacidad vehículo (si no está asignado, y[v,d]==0 → x=0 → carga=0)
def vehicle_cap(m, v):
    return sum( m.demand[i] * sum(m.x[v,j,i] for j in m.N if j!=i)
               for i in m.C ) <= sum(m.y[v,d]*m.vehicle_capacity[v] for d in m.D)
model.veh_cap = Constraint(model.V, rule=vehicle_cap)

# h) capacidad depósito: suma de demandas servidas desde d ≤ cap[d]
# (opcional) pre-extrae las listas para que queden como depot_clients

depot_clients = cientesdeposito

def depot_cap(m, d):
    if not depot_clients[d]:  # si no hay clientes asignados al depósito
        return Constraint.Skip

    return sum(
        m.demand[i] * sum(m.x[v, j, i] for j in m.N if j != i)
        for v in m.V
        for i in depot_clients[d]
    ) <= m.depot_capacity[d]

model.dep_cap = Constraint(model.D, rule=depot_cap)

# i) rango vehículo
def veh_range(m, v):
    return sum(m.distance[i,j]*m.x[v,i,j] for i in m.N for j in m.N if i!=j) \
           <= sum(m.y[v,d]*m.vehicle_range[v] for d in m.D)
model.veh_range = Constraint(model.V, rule=veh_range)

# j) MTZ subtours
def mtz_rule(m, v, i, j):
    if i!=j:
        return (m.u[v,i] - m.u[v,j]
               + m.vehicle_capacity[v]*m.x[v,i,j]
               <= m.vehicle_capacity[v] - m.demand[j])
    return Constraint.Skip
model.mtz     = Constraint(model.V, model.C, model.C, rule=mtz_rule)
model.mtz_bnd = Constraint(model.V, model.C,
                           rule=lambda m,v,i: (m.demand[i], m.u[v,i], m.vehicle_capacity[v]))

# k) satisfacción de demanda parcial (MTZ auxiliar)
def demand_sat(m, v, i):
    return m.u[v,i] >= m.demand[i] * sum(m.x[v,j,i] for j in m.N if j!=i)
model.demand_sat = Constraint(model.V, model.C, rule=demand_sat)

assign_vehicle= vehicles_per_depot
# Restricción: solo permitir y[v,d] si está en assign_vehicle[d]
def respect_vehicle_assignment(m, v, d):
    if v not in assign_vehicle.get(d, []):
        return m.y[v, d] == 0  # No permitido
    return Constraint.Skip
model.vehicle_assign_limit = Constraint(model.V, model.D, rule=respect_vehicle_assignment)

In [11]:
import csv
import os
from pyomo.environ import value

def print_and_save_routes(model, output_file="solutions/verificacion_caso3.csv"):
    # Crear la carpeta 'solutions' si no existe
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Abrir el archivo CSV para escribir
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Escribir la cabecera del CSV
        writer.writerow([
            "VehicleId", "DepotId", "InitialLoad", "RouteSequence", 
            "ClientsServed", "DemandsSatisfied", "TotalDistance", "TotalTime", "FuelCost"
        ])
        
        # Imprimir el costo total
        print(f"\n==== Costo óptimo total del plan de rutas: $ {value(model.obj):.3f}")
        
        # Recorrer cada vehículo
        for v in model.V:
            print(f"\n>>> Ruta del vehículo {v}:")

            # Encontrar el depósito asignado y su capacidad
            depot = None
            for d in model.D:
                if value(model.y[v, d]) > 0.5:
                    depot = d
                    break
            
            if depot is None:
                print("Vehículo no asignado a ningún depósito.")
                continue
            
            # Inicializar variables para la ruta
            current_node = depot
            route = [current_node]
            visited = set()
            deliveries = {}
            total_load = 0
            total_distance = 0
            
            # Reconstruir la ruta del vehículo
            while True:
                next_node = None
                for j in model.N:
                    if j != current_node and value(model.x[v, current_node, j]) > 0.5:
                        next_node = j
                        break
                
                if next_node is None or next_node in visited:
                    break
                
                # Si es un cliente, actualizar la carga y las entregas
                if next_node in model.C:
                    deliveries[(current_node, next_node)] = demand[next_node]
                    total_load += demand[next_node]
                
                route.append(next_node)
                visited.add(next_node)
                current_node = next_node

                # Calcular la distancia total recorrida
                total_distance += value(model.distance[route[-2], route[-1]])

            # Calcular el uso del rango operativo
            vehicle_range_val = vehicle_range[v]
            porcentaje = (total_distance / vehicle_range_val) * 100 if vehicle_range_val > 0 else 0

            # Crear la secuencia de clientes atendidos
            clients_served = len(deliveries)
            demands_satisfied = '-'.join(str(demand[i]) for i in route if i in model.C)

            # Crear la secuencia de rutas (como una cadena separada por guiones)
            route_sequence = '-'.join(route)

            # Calcular el costo de combustible (distancia * precio por km)
            fuel_cost = total_distance * precio_km

            # Imprimir los resultados
            if len(route) > 1:
                print(f" - Ruta: {' → '.join(route)}")
                print(f" - Carga total transportada: {total_load}/{vehicle_capacity[v]} unidades")
                print(f" - Distancia total recorrida: {total_distance:.3f} km")
                print(f" - Uso del rango operativo: {porcentaje:.1f}%")
                print(f' - Costo del transporte: $ {fuel_cost:.2f}')

                print(" - Entregas entre nodos:")
                for (i, j), qty in deliveries.items():
                    print(f"    De {i} → {j}: {qty} unidades")
            else:
                print("Vehículo no utilizado.")

            # Guardar los resultados en el CSV
            writer.writerow([
                f"VEH{v[1:]}", depot, 
                total_load, route_sequence, 
                clients_served, demands_satisfied, 
                total_distance, 0, fuel_cost
            ])



In [12]:
from amplpy import modules

solver_name = "gurobi"
solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
solver.options['TimeLimit'] = 500
solver.options['MIPGap']   = 0.01

res = solver.solve(model, load_solutions=True, tee=True)

from pyomo.opt import TerminationCondition
if res.solver.termination_condition not in (
    TerminationCondition.optimal,
    TerminationCondition.feasible):
    raise RuntimeError(f"No factible/óptimo: {res.solver.termination_condition}")


# Llamar la función para imprimir y guardar las rutas
print_and_save_routes(model)

Gurobi 12.0.1:   lim:time = 500
  mip:gap = 0.01

==== Costo óptimo total del plan de rutas: $ 1051805.169

>>> Ruta del vehículo V1:
 - Ruta: CD4 → C7 → CD4
 - Carga total transportada: 12/132 unidades
 - Distancia total recorrida: 2.370 km
 - Uso del rango operativo: 1.6%
 - Costo del transporte: $ 7186.08
 - Entregas entre nodos:
    De CD4 → C7: 12 unidades

>>> Ruta del vehículo V2:
 - Ruta: CD4 → C67 → C74 → C69 → C20 → C41 → C82 → C63 → C4 → C47 → C66 → C58 → CD1
 - Carga total transportada: 132/136 unidades
 - Distancia total recorrida: 32.280 km
 - Uso del rango operativo: 16.5%
 - Costo del transporte: $ 97876.19
 - Entregas entre nodos:
    De CD4 → C67: 12 unidades
    De C67 → C74: 12 unidades
    De C74 → C69: 12 unidades
    De C69 → C20: 12 unidades
    De C20 → C41: 12 unidades
    De C41 → C82: 12 unidades
    De C82 → C63: 12 unidades
    De C63 → C4: 12 unidades
    De C4 → C47: 12 unidades
    De C47 → C66: 12 unidades
    De C66 → C58: 12 unidades

>>> Ruta del ve

In [13]:
rutas_optimas = []
for v in model.V:
    for i in model.N:
        for j in model.N:
            if i != j and value(model.x[v,i,j]) > 0.5:
                rutas_optimas.append((v, i, j))


Se hizo el mapa frente a cada vehículo ya que no se logro graficar con tos los vehiculos y todos los clientes a través de folium.

In [14]:
import folium
import openrouteservice
from collections import defaultdict
from IPython.display import display
import itertools

API_KEY = '5b3ce3597851110001cf6248781c341d9b2c47b4b5f6ce22fb428092'
client = openrouteservice.Client(key=API_KEY)

colores = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'black', 'pink', 'gray', 'cyan']
vehiculos = sorted(set(v for v, _, _ in rutas_optimas))
vehiculo_a_color = dict(zip(vehiculos, itertools.cycle(colores)))

labels = list(model.N.data())
coords = lugares
label_to_coord = dict(zip(labels, coords))

# rutas para cada vehiculo
rutas_por_vehiculo = defaultdict(list)
for v, i, j in rutas_optimas:
    rutas_por_vehiculo[v].append((i, j))

for v in vehiculos:
    rutas = rutas_por_vehiculo[v]

    # grafo para poder saber donde comienza y termina
    sucesor = {i: j for i, j in rutas}
    predecesores = {j: i for i, j in rutas}

    # saber nodo inicial de cada una de las rutas para cada uno de vehiculos
    posibles_inicios = set(sucesor.keys()) - set(predecesores.keys())
    if not posibles_inicios:
        posibles_inicios = [rutas[0][0]]

    origen = list(posibles_inicios)[0]
    ruta_ordenada = [origen]
    while ruta_ordenada[-1] in sucesor:
        siguiente = sucesor[ruta_ordenada[-1]]
        if siguiente in ruta_ordenada:
            break
        ruta_ordenada.append(siguiente)

    m = folium.Map(location=[4.65, -74.1], zoom_start=11)

    for nodo in ruta_ordenada:
        coord = label_to_coord[nodo]
        folium.Marker(location=(coord[1], coord[0]), popup=nodo).add_to(m)

    color = vehiculo_a_color[v]
    for i, j in zip(ruta_ordenada[:-1], ruta_ordenada[1:]):
        coord_i = label_to_coord[i]
        coord_j = label_to_coord[j]
        try:
            ruta = client.directions(
                coordinates=[coord_i, coord_j],
                profile='driving-car',
                format='geojson'
            )
            folium.GeoJson(
                ruta,
                name=f"{v}: {i}->{j}",
                tooltip=f"{v}: {i} → {j}",
                style_function=lambda x, col=color: {'color': col, 'weight': 4, 'opacity': 0.8}
            ).add_to(m)
        except Exception as e:
            print(f"Error en ruta {i} → {j}: {e}")

    # Mostrar mapa
    print(f"Ruta de {v}: {ruta_ordenada}")
    display(m)


Ruta de V1: ['CD4', 'C7']


Ruta de V10: ['CD7', 'C34', 'C15', 'C84', 'C89', 'C73', 'C23', 'C59', 'C56']


Ruta de V11: ['CD5', 'C51', 'C60', 'C46', 'C76', 'C65', 'C42', 'C19', 'C17', 'C28', 'C3']


Ruta de V12: ['CD8', 'C44', 'C62', 'C78']


Ruta de V13: ['CD10', 'C53', 'C83', 'C80', 'C52', 'C86', 'C25']


...

Ruta de V5: ['CD7', 'C24', 'C35', 'C12', 'C72', 'C14', 'C90', 'C10', 'C43', 'C39']


Ruta de V6: ['CD6', 'C85', 'C30', 'C57', 'CD11']


Ruta de V7: ['CD5', 'C45', 'C18', 'C16', 'C48']


Ruta de V8: ['CD12', 'C22', 'C36', 'C88', 'C8', 'C70']


Ruta de V9: ['CD9', 'C61', 'C49', 'C29', 'C21', 'C9']
